In [51]:
from IPython.display import display
import pandas as pd
import re
from bs4 import BeautifulSoup as b_s
import pandas as pd
import urllib.request as req
import requests
import os
from datetime import datetime as dt
import time
from googleapiclient.discovery import build
import random


##### 오늘 날짜 출력

In [157]:
today = dt.today().strftime('%Y-%m-%d-%A')
today

'2021-07-12-Monday'

##### 오늘 날씨 출력 (네이버 크롤링)

    기상청이나 공공데이터 크롤링 해보신 분에게 직접 물어봐서 해보겠습니다. 제가 할땐 정말 안되더군요

In [161]:
def today_weather():
    html = requests.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EB%82%A0%EC%94%A8')
    soup = b_s(html.text,'html.parser')
    Wdata = soup.find('div', {'class':'weather_box'})
    find_lc = Wdata.find('span',{'class':'btn_select'}).text
    print('현재위치 : '+find_lc,'℃' )
    find_ct = Wdata.find('span',{'class':'todaytemp'}).text
    print("현재온도 : "+find_ct,'℃')
    find_ht = Wdata.select_one('span.max>span.num').text
    print("최고온도 : "+find_ht,'℃')
    find_lt = Wdata.select_one('span.min>span.num').text
    print('최저온도 : '+find_lt,'℃')
    find_ft = Wdata.select_one('span.sensible>em>span.num').text
    print('체감온도 : '+find_ft,'℃')
    find_dt = Wdata.select_one('div.today_area._mainTabContent > div.main_info > div > ul > li:nth-child(1) > p').text
    print(find_dt)
    find_am_hm = Wdata.select_one('li>span.point_time.morning>span.rain_rate>span.num').text
    print('오전강수확률 : '+find_am_hm,'%')
    find_pm_hm = Wdata.select_one('li>span.point_time.afternoon>span.rain_rate>span.num').text
    print('오후강수확률 : '+find_pm_hm,'%')

##### 내일 날씨 출력

In [164]:
def tomorrow_weather():
    html = requests.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EB%82%A0%EC%94%A8')
    soup = b_s(html.text,'html.parser')
    Wdata = soup.find('div', {'class':'weather_box'})
    find_t_ct = Wdata.select_one('div.main_info.morning_box>p.info_temperature>span.todaytemp').text
    print("내일 날씨 : "+find_t_ct,'℃' )
    find_t_dt = Wdata.select_one('div.main_info.morning_box>div.info_data>ul.info_list>li>p').text
    print(find_t_dt)
    find_t_am_hm = Wdata.select_one('li>span.point_time.morning>span.rain_rate.wet>span.num').text
    print('내일강수확률 : ',find_t_am_hm,'%')

##### 미세먼지

In [166]:
def dust():
    html = requests.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EB%82%A0%EC%94%A8')
    soup = b_s(html.text,'html.parser')
    Ddata = soup.find('div', {'class':'weather_box'})
    find_du = Ddata.select_one('dl.indicator>dd>span.num').text
    ev_du = int(find_du.strip('㎍/㎥'))

    if ev_du <= 30:
        ev_dust = "좋음"
    elif 30 <ev_du<= 80:
        ev_dust = "보통"
    elif 80 <ev_du<= 150:
        ev_dust = "나쁨"
    else:
        ev_dust = "매우나쁨"

    print("미세먼지 : " + find_du, ev_dust)

    find_M_du = Ddata.select('dl.indicator>dd>span.num')[1].text
    ev_M_du = int(find_M_du.strip('㎍/㎥'))

    if ev_M_du <= 15:
        ev_M_dust = "좋음"
    elif 15 <ev_M_du<= 50:
        ev_M_dust = "보통"
    elif 51 <ev_M_du<= 100:
        ev_M_dust = "나쁨"
    else:
        ev_M_dust = "매우나쁨"

    print("초미세먼지 : " + find_M_du, ev_M_dust)

##### 코로나

In [180]:
def corona19():
    html = requests.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%BD%94%EB%A1%9C%EB%82%98')
    soup = b_s(html.text,'html.parser')
    html1 = requests.get('http://ncov.mohw.go.kr/')
    soup1 = b_s(html1.text,'html.parser')
    status = soup1.select_one('div.liveNumOuter>h2>a').text
    status = status.split('자세히')[0]
    # type(status)
    status = status.split(',')
    status
    print(status)
    tsct = soup1.select_one('div.datalist>ul>li>span.subtit').string
    tsctn = soup1.select_one('div.datalist>ul>li>span.data').string
    print(tsct,':', tsctn)
    find_cf = soup.select_one('ul>li.info_01>p.info_num').text
    print('누적환자 : ',find_cf,'명')

##### 국내증시

In [193]:
def popular_stock():
    req = requests.get('https://finance.naver.com/sise/')
    html = req.text
    soup = b_s(html,'html.parser')

    stock_names = soup.select('ul.lst_pop>li>a')
#     for stock_name in stock_names:
#         print(stock_name.text)

    stock_costs = soup.select('ul.lst_pop>li>span')
#     for stock_cost in stock_costs:
#         print(stock_cost.text)
    stock_costs
    
    stock_vars = soup.select('ul.lst_pop>li>em.bu_p>span.blind')
#     for stocks_var in stocks_vars:
#         print(stocks_var.string)

    stock = pd.DataFrame([stock_names,stock_costs,stock_vars])
    stock = stock.transpose()
    stock = stock.rename(columns={0:'종목명',1:'현재가',2:'변화'})
    return display(stock)

In [194]:
popular_stock()

,종목명,현재가,변화
0,[삼성전자],"[79,400]",[하락]
1,[카카오게임즈],"[84,500]",[상승]
2,[카카오],"[160,500]",[하락]
3,[현대차],"[226,000]",[하락]
4,[SK하이닉스],"[119,500]",[하락]
5,[휴마시스],"[25,550]",[상승]
6,[씨젠],"[88,000]",[하락]
7,[한국조선해양],"[124,500]",[하락]
8,[ARIRANG 고배당주],"[12,180]",[하락]
9,[TIGER 200 IT],"[38,345]",[하락]


###### 뉴스

In [2]:
def NEWS():
    url = 'https://news.naver.com/'
    headers = {'User-Agent':'Mozilla/5.0 (Windosws NT 6.3; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    res = requests.get(url,headers=headers)
    news_soup = b_s(res.text,'html.parser')
    headline_titles = news_soup.select('div.hdline_news>ul.hdline_article_list>li>div.hdline_article_tit')
    headline_title = []
    for title in headline_titles:
        headline_title.append(title.text.strip())
    headline_title

    headline_links = news_soup.select('div.hdline_news>ul.hdline_article_list>li>div.hdline_article_tit>a')
    headline_link = []
    for link in headline_links:
        headline_link.append("https://news.naver.com"+link.get('href'))
    headline_link

    headline_news = pd.DataFrame([headline_title,headline_link]).T
    headline_news.rename(columns = {0:'Headline News Title',1:'Link'},inplace=True)
    # headline_news['Link'] = headline_news['Link'].apply(lambda x : make_hyper(x))
#     folder_path = os.getcwd()
#     headline_news.to_excel('Today News.xlsx')
#     os.startfile(folder_path)



    # def make_hyper(value):
    #     return '=HYPERLINK("%s")' % (value.format(value))
    # def make_clickable(val):
    #     return '<a href="{}">{}</a>'.format(val,val)
    return headline_news.style.format(make_clickable)

NEWS()

NameError: name 'requests' is not defined

##### food

In [83]:
rain_foods = "부대찌개,아구찜,해물탕,칼국수,수제비,짬뽕,우동,치킨,국밥,김치부침개,두부김치,파전".split(',')
mdhigh_foods = "콩나물국밥,고등어,굴,쌀국수,마라탕".split(',')
hightemp_foods ="냉면,삼계탕,밀면,소바,스파게티,베이글,치킨".split(',')

In [84]:
# 네이버 인증
# https://developers.naver.com/apps
# 해당 사이트에서 로그인 후 "Cliend ID"와 "Client Secret"을 얻어오세요
ncreds = {
    "client_id": "IS7Lmhp7lT6z4uFNjEG5",      
    "client_secret" : "8LtHME2zTz"
}
nheaders = {
    "X-Naver-Client-Id" : ncreds.get('IS7Lmhp7lT6z4uFNjEG5'),
    "X-Naver-Client-Secret" : ncreds.get('8LtHME2zTz')
}

In [85]:
html = requests.get('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=%EB%82%A0%EC%94%A8')
soup = b_s(html.text,'html.parser')
Wdata = soup.find('div', {'class':'weather_box'})
find_ft = Wdata.select_one('span.sensible>em>span.num').text
foods_list = None

if int(find_ft) >=   25:
    foods_list = random.sample(hightemp_foods,k=len(hightemp_foods))
elif 19 <= int(find_ft) < 25:
    foods_list = random.sample(mdhigh_foods, k=len(mdhigh_foods))
elif int(find_ft) < 19:
    foods_list = random.sample(rain_foods, k=len(rain_foods))
else:
    foods_list = ['']
    
foods_list

['삼계탕', '소바', '베이글', '스파게티', '밀면', '냉면', '치킨']

In [94]:
import urllib
# urllib.parse.quote(query) URL에서 검색어를 인코딩하기 위한 라이브러리

# 네이버 지역 검색 주소
naver_local_url = "https://openapi.naver.com/v1/search/local.json?"

# 검색에 사용될 파라미터
# 정렬 sort : 리뷰순(comment)
# 검색어 query : 인코딩된 문자열
params_format = "sort=comment&query="

# 위치는 사용자가 사용할 지역으로 변경가능
location = "이대역"

# 추천된 맛집을 담을 리스트
recommands = []
for food in foods_list:
    # 검색어 지정
    query = location + " " + food + " 맛집"
    # 지역검색 요청 파라메터 설정
    params = "sort=comment" \
              + "&query=" + query \
              + "&display=" + '5'
    
    # 검색
    # headers : 네이버 인증 정보
    res = requests.get(naver_local_url + params, headers=nheaders)
    
    # 맛집 검색 결과
    result_list = res.json().get('items')

    # 경우 3 처리
    # 맛집 검색 결과에서 가장 상위 3개를 가져옴
    if 21 <= int(find_ft) <= 30:
        for i in range(0,3):
            recommands.append(result_list[i])
        break
    # 경우 1,2 처리
    # 해당 음식 검색 결과에서 가장 상위를 가져옴
    if result_list:
        recommands.append(result_list[0])
        # 3개를 찾았다면 검색 중단
        if len(recommands) >= 3:
            break

TypeError: 'NoneType' object is not subscriptable

In [87]:
recommands

[]

##### 길찾기

# 모두 출력

In [203]:
print('#'*50)
print('        ',today,"의 리마인더입니다.")
print('#'*50,'\n\n')
print('1.날씨')
print('*'*50)
print("                        오늘")
print('-'*50)
today_weather()
print('*'*50)
print("<미세먼지>")
print('-'*50)
dust()
print('\n')
print('-'*50)
print("                 내일 날씨입니다.")
print('-'*50)
tomorrow_weather()
print('\n')
print('2.코로나19 현재 상황')
print('*'*50,'\n\n')
corona19()
print('3.인기 국내증시 현재 상황')
print('*'*50)
popular_stock()

print('4. News')
NEWS()

##################################################
         2021-07-12-Monday 의 리마인더입니다.
################################################## 


1.날씨
**************************************************
                        오늘
--------------------------------------------------
현재위치 : 경기도 김포시 김포본동 ℃
현재온도 : 25 ℃
최고온도 : 32 ℃
최저온도 : 24 ℃
체감온도 : 27 ℃
흐림, 어제보다 1˚ 높아요
오전강수확률 : 60 %
오후강수확률 : 20 %
**************************************************
<미세먼지>
--------------------------------------------------
미세먼지 : 10㎍/㎥ 좋음
초미세먼지 : 9㎍/㎥ 좋음


--------------------------------------------------
                 내일 날씨입니다.
--------------------------------------------------
내일 날씨 : 25 ℃
흐림
내일강수확률 :  60 %


2.코로나19 현재 상황
************************************************** 


['환자 현황(7.11. 00시 기준', " '20.1.3. 이후 누계)"]
국내발생 : 1,280
누적환자 :  168,046 명
3.인기 국내증시 현재 상황
**************************************************


,종목명,현재가,변화
0,[삼성전자],"[79,400]",[하락]
1,[SK하이닉스],"[119,500]",[하락]
2,[씨젠],"[88,000]",[하락]
3,[코오롱인더우],"[33,750]",[하락]
4,[카카오],"[160,500]",[하락]
5,[한국조선해양],"[124,500]",[하락]
6,[두산중공업],"[26,100]",[상승]
7,[엑세스바이오],"[34,400]",[하락]
8,[DL이앤씨우],"[80,000]",[하락]
9,[우리금융지주],"[11,150]",[하락]


4. News?


,Headline News Title,Link
0,與 “부끄러운 줄 알라”…野 김재원 “선거인단 돼 달라며”,https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=025&aid=0003117307
1,신규 확진자 1100명대 예상…오늘부터 수도권 거리두기 4단계,https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=025&aid=0003117404
2,"아이티 영부인, 피격 후 첫 육성…“말할 기회도 없이 총 난사”",https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=104&oid=020&aid=0003369379
3,G20 '디지털세' 합의···131개국서 稅 177조 추가로 걷힐 듯,https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=101&oid=011&aid=0003935087
4,이재명 '김건희 검증' 거리두기에 與추격자들 일제히 공세(종합),https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=001&aid=0012517664


In [411]:
# def auto_news():
#     URL = 'https://news.naver.com/'
#     res = req.urlopen(URL).read()
#     news_soup = b_s(res,'html.parser')
    
#     #list 생성
#     news_list = []
#     new_news = []
#     news_link = []
    
#     #헤드라인 
#     news_title = news_soup.select('div#today_main_news>div.heline|_news>ul>li>div.hdline_article_tit')
#     news_links = news_soup.select('div#today_main_news>div.hdline_news>ul>li>div.hdline_article_tit>a')
    
#     #리스트별 추가
#     for news in news_title:
#         new_news.append(news.text.strip())
        
#     for link in news_link:
#         news_link.append("https://naver.com"+link.get('href'))
    
#     #담기    
#     for i in range(len(new_news)):
#         news_list.append([[new_news[i],news_link[i]]])

#     #시간별
#     time = dt.today().strftime('%Y-%m-%d')
    
#     #DF
#     news_data = pd.DataFrame(news_list)
#     news_data.columns = [time+'Title',"Link"]
#     news_data.to_csv('news_headline_today.csv',mode = 'a',encoding='euc-kr')


# def find_News():
#     #엑셀 저장용
#     date=str(dt.now())

#     query = input('검색 키워드를 입력하세요 : ')
#     news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : '))
#     query = query.replace(' ', '+')

#     #URL 확인
#     news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'
#     req = requests.get(news_url.format(query))
#     soup = b_s(req.text, 'html.parser')


#     news_dict = {}
#     idx = 0
#     cur_page = 1

#     print()
#     print('크롤링 중...')

#     while idx < news_num:
#     ### 네이버 뉴스 웹페이지 구성이 바뀌어 태그명, class 속성 값 등을 수정함(20210126) ###

#         table = soup.find('ul',{'class' : 'list_news'})
#         li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
#         area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
#         a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]

#         for n in a_list[:min(len(a_list), news_num-idx)]:
#             news_dict[idx] = {'title' : n.get('title'),
#                               'url' : n.get('href') }
#             idx += 1

#         cur_page += 1

#         pages = soup.find('div', {'class' : 'sc_page_inner'})
#         next_page_url = [p for p in pages.find_all('a') if p.text == str(cur_page)][0].get('href')

#         req = requests.get('https://search.naver.com/search.naver' + next_page_url)
#         soup = b_s(req.text, 'html.parser')

#     print('크롤링 완료')

#     print('데이터프레임 변환')
#     news_df = pd.DataFrame(news_dict).T

#     folder_path = os.getcwd()
#     xlsx_file_name = '네이버뉴스_{}_{}.xlsx'.format(query, date)

#     news_df.to_excel(xlsx_file_name)

#     print('엑셀 저장 완료 | 경로 : {}\\{}'.format(folder_path, xlsx_file_name))
#     os.startfile(folder_path)